In [3]:
import database
import database.chat_history

database.chat_history.delete(user_id="test")

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000203'), 'opTime': {'ts': Timestamp(1727257181, 9), 't': 515}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1727257181, 9), 'signature': {'hash': b'J^\xc7#\xb1\x83\xa3\xba\xf5\xb3+\x11p0Y\xbc.\x90\xdf\xaf', 'keyId': 7366617972116291585}}, 'operationTime': Timestamp(1727257181, 9)}, acknowledged=True)